# e3ferminet
E(3)-equivariant neural network ansatz for molecular VMC calculations

## Single atom wavefunction
Let's first study a single atom of atomic number $Z$. For $n$ electrons, the Hamiltonian is
$$
\hat H = -\frac{1}{2} \sum_i \nabla_i^2 - \sum_i \frac{Z}{r_i} + \sum_{i<j} \frac{1}{|\mathbf r_i - \mathbf r_j|}.
$$
We'll parameterize the multi-electron wavefunction $\psi(\mathbf r_1, \ldots, \mathbf r_n)$ with a neural network $\phi_\theta(\mathbf r_1, \ldots, \mathbf r_n)$ that is equivariant in each input $\mathbf r_i$, where we obtain $\psi$ by antisymmetrizing $\phi_\theta$:
$$
\psi(\mathbf r_i, \ldots, \mathbf r_n) = \sum_{\sigma \in S(n_\uparrow) \times S(n_\downarrow)} \mathrm{sgn}(\sigma) \phi_\theta(\mathbf r_{\sigma(1)}, \ldots, \mathbf r_{\sigma(n)}).
$$
The sum is over all products of permutations of $n_\uparrow$ spin-up electrons with permutations of $n_\downarrow$ spin-down electrons, where $n_\uparrow + n_\downarrow = n$, where WLOG we label the spin-up electrons as $1, \ldots, n_\uparrow$ and the spin-down electrons as $n_\uparrow + 1, \ldots, n$.


# Molecules with multiple atoms